# MC Code RAG


## Setup


### Hyper parameters


In [ ]:
# p1=0  #Parameter 1 


### Dependencies


In [ ]:
# %pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai chromadb bs4 python-dotenv

### Key from .env


In [31]:
import os
from dotenv import load_dotenv,find_dotenv

# Use this line of code if you have a local .env file
load_dotenv(find_dotenv()) 

# Or set it like this
# os.environ["OPENAI_API_KEY"] = "sk-..."

# Print this line to double check your API key
# print(os.environ["OPENAI_API_KEY"])

True

### LangSmith(enabled by default)


In [32]:
# Store all the keys in .env
import getpass
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.environ["LANGCHAIN_API_KEY"] #getpass.getpass()
print(os.environ["LANGCHAIN_API_KEY"])

### Import from


In [33]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader, TextLoader, PyPDFLoader, PyPDFium2Loader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document

### Load File


In [ ]:
# Web loader
# loader = WebBaseLoader(
#     web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#     ),
# )
# docs = loader.load()

In [34]:
# Pdf loader
file_path = './data/WMX3API c++ to python list -all.pdf'
# file_path = 'nais2023.pdf'
# loader = PyPDFLoader(file_path)
# loader = PyPDFLoader(file_path, extract_images=True)   #extract images as text as well

loader = PyPDFium2Loader(file_path,  extract_images=False) 
# docs = loader.load_and_split()
docs = loader.load()
print(docs[0])


page_content='No Function C++ Function Python Return\r\nType\r\nReturn Value Class \r\n1 static WMX3APIFUNC ErrorToString(int errCode, wchar_t *pString, unsigned int size); ApiBuffer Class \r\n2 static WMX3APIFUNC ApiLogToString(unsigned char *pLogData, unsigned int logDataSize, char *pString, unsigned int size); def ApiLogToString(pLogData, logDataSize) string API log string ApiBuffer Class \r\n3 static WMX3APIFUNC ApiLogToString(unsigned char *pLogData, unsigned int logDataSize, wchar_t *pString, unsigned int size); ApiBuffer Class \r\n4 static WMX3APIFUNC GetLibVersion(int *pMajorVersion, int *pMinorVersion, int *pRevisionVersion, int *pFixVersion); def GetLibVersion() tuple 0 : int - error code ApiBuffer Class \r\n5 bool IsDeviceValid(); def IsDeviceValid() bool Whether the device is\r\nalive \r\nApiBuffer Class \r\n6 WMX3APIFUNC GetVersion(int *pMajorVersion, int *pMinorVersion, int *pRevisionVersion, int *pFixVersion); def GetVersion() tuple 0 : int - error code ApiBuffer Class \

/opt/anaconda3/lib/python3.11/site-packages/pypdfium2/_helpers/textpage.py:81: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")


In [36]:
len(docs)


30

In [ ]:
# Txt loader
# loader = TextLoader("WMX3UserManual c.txt")
# docs = loader.load()
# docs[0].page_content[:100000]
# docs

### Chunk


In [37]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
splits = text_splitter.split_documents(docs)

### Vectorstore - chroma


In [38]:
embedding_model=OpenAIEmbeddings(model="text-embedding-3-large")   #text-embedding-3-large   #text-embedding-ada-002    #text-embedding-3-small

# If txt vectorstore exists
# if os.path.exists("Vectorstore/chromadb"):
        # vectorstore = Chroma(
                #     embedding_function=embedding_model,
                #     persist_directory="Vectorstore/chromadb",
                #     ) 
# else:
        # Load from chunks and save to disk
        # vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model, persist_directory="Vectorstore/chromadb") 

# If pdf vectorstore exists
# vectorstore_path = "Vectorstore/chromadb-pdf"
vectorstore_path = "Vectorstore/chromadb-pdf-chunk1000"
 # Load from chunks and save to disk
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model, persist_directory=vectorstore_path) 
# if os.path.exists(vectorstore_path):
#         vectorstore = Chroma(
#                     embedding_function=embedding_model,
#                     persist_directory=vectorstore_path,
#                     ) 
# else:
#         # Load from chunks and save to disk
#         vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model, persist_directory=vectorstore_path) 


In [39]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})


In [40]:
retrieved_docs = retriever.invoke("write a code to Move Axis 1 to 20008")
#print(retrieved_docs[0].page_content)
retrieved_docs 

--- Logging error ---
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/site-packages/langchain_core/callbacks/manager.py", line 2004, in _configure
    handler = LangChainTracer(project_name=tracer_project)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/langchain_core/tracers/langchain.py", line 91, in __init__
    self.client = client or get_client()
                            ^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/langchain_core/tracers/langchain.py", line 54, in get_client
    _CLIENT = Client()
              ^^^^^^^^
  File "/opt/anaconda3/lib/python3.11/site-packages/langsmith/client.py", line 485, in __init__
    _validate_api_key_if_hosted(self.api_url, self.api_key)
  File "/opt/anaconda3/lib/python3.11/site-packages/langsmith/client.py", line 274, in _validate_api_key_if_hosted
    raise ls_utils.LangSmithUserError(
langsmith.utils.LangSmithUserError: API key must

[Document(page_content='After the above code is executed, the axis will move to position 10000, and then to\r\nposition 20000.\r\nThe position and velocity plots of the axis during this motion are shown below.\r\nGenerated by 1.8.14\r\n}\r\n//Block execution until motion is finished\r\nerr = wmxlib_cm.motion->Wait(0);', metadata={'page': 201, 'source': './data/WMX3UserManual.pdf', 'start_index': 0}),
 Document(page_content='pos.target = -10000;\r\nwmxlib_CoreMotion->motion->StartMov(&pos);\r\n//Wait until motion is finished\r\nwmxlib_CoreMotion->motion->Wait(0);\r\n//Execute motion to move axis forward 10000\r\npos.target = 10000;\r\nwmxlib_CoreMotion->motion->StartMov(&pos);\r\nThe following plots show the position, velocity, and acceleration when the above code is\r\nexecuted.', metadata={'page': 1080, 'source': './data/WMX3UserManual.pdf', 'start_index': 0}),
 Document(page_content='The position and velocity plots of the axis during this motion are shown below.\r\nerr = wmxlib_cm.mo

### Vectorstore - pinecone


In [ ]:
from langchain_pinecone import PineconeVectorStore
index_name = os.environ["PINECONE_INDEX_NAME"]


In [ ]:
# PineconeVectorStore.delete(delete_all=True)

In [ ]:

docsearch = PineconeVectorStore.from_documents(splits, embedding_model, index_name=index_name)


In [ ]:
docsearch = PineconeVectorStore.from_existing_index(index_name, embedding_model)


In [ ]:
query = "what is the Architecture of wmx3?"
retrieved_docs = docsearch.similarity_search(query,k=8)
retrieved_docs

### Vectorstore - FAISS

In [ ]:
embedding_model=OpenAIEmbeddings(model="text-embedding-3-large")   

from langchain_community.vectorstores import FAISS
vectorstore_path = "Vectorstore/FAISS-pdf-images"
db = FAISS.from_documents(splits, embedding_model) 

In [ ]:
db.save_local(folder_path="Vectorstore/FAISS-pdf-images", index_name="myFaissIndex")

In [ ]:
db = FAISS.load_local(folder_path="Vectorstore/FAISS-pdf-images",embeddings=embedding_model,index_name="myFaissIndex")

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)
docs

### Vectorstore - pathway

### serve Maximum marginal relevance search (MMR) Similarity search by vector ...

In [ ]:
from langchain_community.vectorstores import Qdrant
qdrant = Qdrant.from_documents(
    splits,
    embedding_model,
    path="Vectorstore/Qdrant-pdf-images",  # Local mode with in-memory storage only
    collection_name="my_documents",
)

### Prompt template


In [ ]:
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}

Helpful Answer:"""


custom_rag_prompt = PromptTemplate.from_template(template)


In [ ]:
custom_rag_prompt = hub.pull("rlm/rag-prompt")

### Rag chain


In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)   #gpt-4  #gpt-3.5-turbo

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)


In [ ]:
format_docs


In [ ]:

rag_chain.invoke("write an example code to close wmx3?")

In [ ]:
for chunk in rag_chain.stream("write a sample code to initialize wmx3?"): 
    print(chunk, end="", flush=True)